In [4]:
from google.colab import drive
drive.mount('/content/drive')

ValueError: Mountpoint must not already contain files

## 1.DNN

In [16]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms # Data Preprocessing Library
from torch.utils.data import DataLoader, TensorDataset

from torch.autograd import Variable

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [5]:
data_path = '/content/drive/MyDrive/FLY_AI/KMK/data'
train_dataset = torchvision.datasets.FashionMNIST(root=data_path, train=True, download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.FashionMNIST(root=data_path, train=False, download=True, transform=transforms.ToTensor())


100%|██████████| 26.4M/26.4M [00:01<00:00, 21.2MB/s]


Extracting /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw



100%|██████████| 29.5k/29.5k [00:00<00:00, 341kB/s]


Extracting /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw



100%|██████████| 4.42M/4.42M [00:00<00:00, 6.13MB/s]


Extracting /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw



100%|██████████| 5.15k/5.15k [00:00<00:00, 12.8MB/s]

Extracting /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/drive/MyDrive/FLY_AI/KMK/data/FashionMNIST/raw



In [6]:
train_loader = DataLoader(train_dataset,
                          batch_size=100, shuffle=False)
test_loader = DataLoader(test_dataset,
                         batch_size=100, shuffle=False)

In [12]:
class FashionDNN(nn.Module):
  def __init__(self):
    super(FashionDNN, self).__init__()
    self.fc1 = nn.Linear(784, 256)
    self.drop = nn.Dropout(0.25)
    self.fc2 = nn.Linear(256, 128)
    self.fc3 = nn.Linear(128, 10)

  def forward(self, x):
    x = x.view(-1, 784)
    x = F.relu(self.fc1(x))
    x = self.drop(x)
    x = F.relu(self.fc2(x))
    output = self.fc3(x)

    return output

In [13]:
learning_rate = 0.001
model = FashionDNN().to(device)

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(model)

FashionDNN(
  (fc1): Linear(in_features=784, out_features=256, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [20]:
num_epochs = 5
count = 0

loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

for epoch in range(num_epochs):
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)
    train = Variable(images.view(100, 1, 28, 28)) # 배치 크기 100, 채널 1, 28x28 이미지 크기
    labels = Variable(labels)

    outputs = model(train)
    loss_train = loss(outputs, labels)
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    count += 1

    if not (count % 50):
      total = 0
      correct = 0
      for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels_list.append(labels)
        test = Variable(images.view(100, 1, 28, 28))

        outputs = model(test)
        predictions = torch.max(outputs, 1)[1].to(device)
        predictions_list.append(predictions)
        correct += (predictions == labels).sum()

        total += len(labels)

      accuracy = correct * 100 / total
      loss_list.append(loss_train)
      iteration_list.append(count)
      accuracy_list.append(accuracy)

    if not (count % 500):
      print(f"Iteration: {count}, Loss: {loss_train.data:.4f}, Accuracy: {accuracy:.4f}")

Iteration: 200, Loss: 0.2815, Accuracy: 86.4500


## 2.CNN

### Model Architecture

In [22]:
class FashionCNN(nn.Module):
  def __init__(self):
    super(FashionCNN, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(2) # 풀링에서 이렇게 쓰면 kernel_size=2 & stride는 default(default=kernel_size와 동일값)
    )

    self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
    self.drop = nn.Dropout2d(0.25)
    self.fc2 = nn.Linear(in_features=600, out_features=120)
    self.fc3 = nn.Linear(in_features=120, out_features=10)

  def forward(self, x):
    output = self.layer1(x)
    output = self.layer2(output)
    output = output.view(output.size(0), -1)
    output = self.fc1(output)
    output = self.drop(output)
    output = self.fc2(output)
    output = self.fc3(output)

    return output

- `stride`의 기본값은 **pooling layer**와 **CNN layer**에서 다르다.

### Set Hyperparameter

In [23]:
num_epochs = 5
count = 0
loss_list = []
iteration_list = []
accuracy_list = []

predictions_list = []
labels_list = []

In [24]:
for epoch in range(num_epochs):
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    train = Variable(images.view(100, 1, 28, 28)) # 배치 크기 100, 채널 1, 28x28 이미지 크기
    labels = Variable(labels)

    outputs = model(train)
    loss_train = loss(outputs, labels)
    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    count += 1

    if not (count % 50):
      total = 0
      correct = 0
      for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        labels_list.append(labels)
        test = Variable(images.view(100, 1, 28, 28))

        outputs = model(test)
        predictions = torch.max(outputs, 1)[1].to(device)
        predictions_list.append(predictions)
        correct += (predictions == labels).sum()

        total += len(labels)

      accuracy = correct * 100 / total
      loss_list.append(loss_train) # .data를 해줘야 하나?
      iteration_list.append(count)
      accuracy_list.append(accuracy)

    if not (count % 500):
      print(f"Iteration: {count}, Loss: {loss_train.data:.4f}, Accuracy: {accuracy:.4f}")

Iteration: 500, Loss: 0.4739, Accuracy: 86.6400
Iteration: 1000, Loss: 0.4209, Accuracy: 86.8200
Iteration: 1500, Loss: 0.2954, Accuracy: 87.3200
Iteration: 2000, Loss: 0.3718, Accuracy: 86.7900
Iteration: 2500, Loss: 0.1997, Accuracy: 87.4500
Iteration: 3000, Loss: 0.2638, Accuracy: 87.3100
